# CH1. EDA


In [1]:
# 라이브러리 불러오기

%matplotlib inline

from pathlib import Path

import pandas as pd
import numpy as np
from scipy.stats import trim_mean
from statsmodels import robust
import wquantiles

import seaborn as sns
import matplotlib.pylab as plt

## 데이터 import

In [2]:
try:
    import common
    DATA = common.dataDirectory()
except ImportError:
    DATA = Path().resolve() / 'data'

Define paths to data sets. If you don't keep your data in the same directory as the code, adapt the path names.

In [4]:
AIRLINE_STATS_CSV = DATA / 'airline_stats.csv'
KC_TAX_CSV = DATA / 'kc_tax.csv.gz'
LC_LOANS_CSV = DATA / 'lc_loans.csv'
AIRPORT_DELAYS_CSV = DATA / 'dfw_airline.csv'
SP500_DATA_CSV = DATA / 'sp500_data.csv.gz'
SP500_SECTORS_CSV = DATA / 'sp500_sectors.csv'
STATE_CSV = DATA / 'state.csv'

## 1.3.3 예제: 인구에 따른 살인 비율의 위치 추정
### 평균과 절사평균, 중앙값

In [5]:
# 주 별 인구 및 살인 비율을 담고있는 데이터 state.csv
# 상위 10 개만 살펴보기

state = pd.read_csv(STATE_CSV)
print(state.head(10))

         State  Population  Murder.Rate Abbreviation
0      Alabama     4779736          5.7           AL
1       Alaska      710231          5.6           AK
2      Arizona     6392017          4.7           AZ
3     Arkansas     2915918          5.6           AR
4   California    37253956          4.4           CA
5     Colorado     5029196          2.8           CO
6  Connecticut     3574097          2.4           CT
7     Delaware      897934          5.8           DE
8      Florida    18801310          5.8           FL
9      Georgia     9687653          5.7           GA


파이썬에서 평균과 중간값을 계산하기 위해 데이터 프레임을 위한 pandas 와 

절사평균 사용을 위해 scipy.stats의 trim_mean 함수를 사용한다.

**절사평균 : 값들을 크기순으로 정렬한 수, 양 끝(가장 크거나 가장 작거나)에서 일정 개수의 값들을 삭제한 뒤 남은 값들을 가지고 구한 평균이다. 극단값의 영향을 제거하기 때문에 선호하는경우가 있다.**

In [6]:
# 인구 평균

state = pd.read_csv(STATE_CSV)
print(state['Population'].mean())

6162876.3


In [7]:
# 인구의 절사 평균

print(trim_mean(state['Population'], 0.1))

4783697.125


In [8]:
# 인구의 중앙값

print(state['Population'].median())

4436369.5


### 가중평균과 가중중간값

가중 평균을 구하기 위해 넘파이를 사용.

가중 중간값을 구하기 위해서는 wquantiles 라는 패키지를 따로 사용.

In [9]:
# 살인 비율

print(state['Murder.Rate'].mean())

4.066


In [10]:
# 인구에 따른 가중치를 준 살인 비율의 평균 

print(np.average(state['Murder.Rate'], weights = state['Population']))

4.445833981123393


In [11]:
# 인구에 따른 가중치를 준 살인 비율의 중간값

print(wquantiles.median(state['Murder.Rate'], weights = state['Population']))

4.4


실제로 그냥 mean 을 구했을 때보다 절사평균을 사용하거나, 인구에 따른 가중치를 준 가중 평균 등을 사용했을 때 본래 중간값과 거의 비슷하게 나오는 것을 볼 수 있었다.

# 변이(variability) 추정

같은 주별 인구의 변이 추정값을 구해본다.

In [12]:
# 데이터프레임 살펴보기

state.head()

,State,Population,Murder.Rate,Abbreviation
0,Alabama,4779736,5.7,AL
1,Alaska,710231,5.6,AK
2,Arizona,6392017,4.7,AZ
3,Arkansas,2915918,5.6,AR
4,California,37253956,4.4,CA


### 표준편차

In [13]:
print(state['Population'].std())

6848235.347401142


### 사분위범위

In [ ]:
print(state['Population'].quantile(0.75) - state['Population'].quantile(0.25))

### 중위절대편차(MAD: statsmodel의 strong.scale.mad 함수 이용)

표준편차는 MAD의 거의 두 배가 된다. 표준편차는 특잇값에 민감하기 때문에 그럴 수 있다.

In [ ]:
print(robust.scale.mad(state['Population']))

아래의 경우 R에서 MAD의 척도가 평균과 같은 척도를 갖도록 보정되는 과정이다.

In [ ]:
print(abs(state['Population'] - state['Population'].median()).median() / 0.6744897501960817)

## 백분위수와 박스플롯
주별 살인율의 백분위수를 구해본다.

### 백분위수

In [ ]:
state['Murder.Rate'].quantile([0.05, 0.25, 0.5,0.75, 0.95])

중간값 0.5는 인구의 중간값이 400만명 정도라고 했을 때 살인 비율로, 인구 10만 명당 4건의 살인이 있다는 것을 알 수 있다.

In [ ]:
# 예쁘게 그리기 (원본 주피터노트북 그대로 가져옴)

percentages = [0.05, 0.25, 0.5, 0.75, 0.95]
df = pd.DataFrame(state['Murder.Rate'].quantile(percentages))
df.index = [f'{p * 100}%' for p in percentages]
print(df.transpose())     # 가로로 보기 위해 행렬을 전치시켰다!

### 박스플롯

주별 인구 분포를 나타내는 박스플롯을 그려보자

In [ ]:
sns.boxplot(data=state['Population'])

너무 못생겼다. 뀨잉뀨잉.

실제로 주어진 코드에서는 이렇게 작성한다.

In [ ]:
ax = (state['Population']/1_000_000).plot.box(figsize=(3, 4))  # 스케일 조정 및 그래프 사이즈 변경
ax.set_ylabel('Population (millions)') # 축 레이블

plt.tight_layout() 
plt.show()

그래프를 보면

중간값이 약 5(500만)에 걸쳐있고 인구수가 극단적으로 높은 이상치가 바로 있음을 알 수 있다. 기본 설정상 휘스커가 사분위수의 1.5배까지만 표시되도록 되어있다.

실제로 박스플롯에서 outlier로 표현되는 경우는 최댓값 즉 제 3 사분위수 + (사분위범위Q3-Q1)* 1.5 를 더한 위치를 넘어가거나, 최솟값 즉 제 1사분위수 - IQR * 1.5 한 위치에 닿지 않는 값이다.

### 각 y값을 가져오기(get_ydata())

In [ ]:
# 각 값의 y값을 가져와보자 (출처: https://blog.naver.com/faithfuljk/222649208086)

# fig, ax = plt.subplots()

# notch파라미터 True로 하면 중앙값의 95%신뢰 구간을 노치 형태로 표시한다!
box = ax.boxplot(state['Population']/1_000_000, notch = True)

whiskers = [item.get_ydata() for item in box['whiskers']]
print('whiskers:', whiskers)

# 각각 Q1,Q3,max,min 값

In [ ]:
medians = [item.get_ydata() for item in box['medians']]
print('medians:', medians)

# 중앙값에 대한 정보를 갖는 Line2D 인스턴스의 리스트.

In [ ]:
fliers = [item.get_ydata() for item in box['fliers']]
print('fliers:', fliers)

# 수염 범위 밖의 데이터 포인트를 반환한다.
# 만약 인구수 말고도 다른 피처들도 함께 했으면 함께 한 만큼 어레이가 여럿 나올 것

## 도수분포표와 히스토그램
도수분포표는 변수위 범위를 동일한 크기의 구간으로 나눈 다음, 각 구간마다 몇 개의 변수값이 존재하는지 보기 위해 사용된다.

In [ ]:
# 주별 인구의 도수분포표 만들기 with cut, value_counts()


"""
pd.cut()은 해당 데이터를 각 구간에 맵핑하는 시리즈를 만든다.
10을 줬기 때문에 10개의 구간으로 나눈다.
"""

population_bin10 = pd.cut(state['Population'], 10)
print(population_bin10.value_counts())

In [ ]:
# Table 1.5 주 별 인구 도수분포표 만들기

population_bin10.name = 'binnedPopulation' 
df = pd.concat([state, population_bin10], axis=1) # 주와 concat함
df = df.sort_values(by='Population') # 인구수 별로 sorting

groups = []
for group, subset in df.groupby(by='binnedPopulation'):
    groups.append({
        'BinRange': group,
        'Count': len(subset),
        'States': ','.join(subset.Abbreviation)
    })
print(pd.DataFrame(groups))

이를 판다스에 있는 매소드로 히스토그램을 그린다. 

구간 개수를 정의하기 위해 bins를 사용한다.

In [ ]:
ax = (state['Population'] / 1_000_000).plot.hist(figsize=(4, 4)) # 스케일 조정 및 그래프 사이즈
ax.set_xlabel('Population (millions)') # 축 레이블

plt.tight_layout()
plt.show()

In [ ]:
# 여기서 살펴보는 이... df의 모습...
df

히스토그램은 다음과 같은 정보들을 담고 있다.

- 그래프에 빈 구간들이 있을 수 있다.
- 구간은 동일한 크기를 갖는다.
- 구간의 수(혹은 구간의 크기)는 사용자가 결정할 수 있다.
- 빈 구간이 있지 않은 이상 막대 사이는 공간 없이 서로 붙어있다.

## 밀도 추정